### Training a Network with Ignite

In [17]:
import os
import configparser

import torchvision
from torchvision.transforms import Compose
import torchvision.transforms.transforms as T

import torch
from torch.utils.data  import DataLoader

ROOT_PATH = os.path.abspath(os.pardir)

#### Read configuration

In [2]:
dir_path = os.path.join(ROOT_PATH, 'config')
file_path = os.path.join(dir_path, 'ExperimentoA.ini')

config = configparser.ConfigParser()
config.read(file_path)

['..\\config\\ExperimentoA.ini']

In [12]:
# Configuration file
print(f"Config file for experiment: {config['general'].get('experiment_name')}")

data_cfg = config['dataset']
data_args = 
model_cfg = config['model']

Config file for experiment: None


#### Data pipeline
Define modular components

In [20]:
from segwork.data import DroneDataset
from segwork.registry import ConfigurableRegistry

dataset_reg = ConfigurableRegistry(
    class_key='dataset',
    additional_args=['transform', 'target_transform'],
    initial_registry=dict(

        # Aerial drone dataset
        drone = dict(
            dataset = DroneDataset,
            _default_kwargs = dict(
                root = os.path.join(ROOT_PATH, 'data', 'semantic_drone_dataset'),
                pil_target = False,
            )
        ), 

        # VOC Segmentation dataset
        # voc = dict(
        #     dataset = torchvision.datasets.VOCSegmentation,
        #     _default_kwargs = dict(
        #          root = os.path.join(ROOT_PATH, 'data', 'PASCALVOC') 
        #     )
        # )
    ),
)

In [23]:
dataset_reg.register(
    dict(
        dataset = torchvision.datasets.VOCSegmentation,
        _default_kwargs = dict(
            root = os.path.join(ROOT_PATH, 'data', 'PASCALVOC') 
        )
))

TypeError: _decorator() missing 1 required positional argument: 'cls'

In [22]:
dataset_reg.get_instance('voc')

KeyError: 'voc'

In [24]:

dataset_cls = dataset_reg[data_cfg['id']]

train_transform = dict(transform = Compose([T.ToTensor()]), target_transform = Compose([T.ToTensor()]))

In [13]:
DATA_DIR = os.path.join(os.pardir, db_cfg['data_path'])
dataset_cls = dataset_reg[db_cfg['id']]

orig_height, orig_width = (db_cfg['orig_height'], db_cfg['orig_width'])
height, width = (db_cfg['resized_height'], db_cfg['resized_width'])              # Make all features divisble by 2 

train_image_transformation = Compose([ T.ToTensor(), T.Resize(size=(height, width)) ])
train_label_transformation = Compose([ T.Resize(size=(height, width))])

train_transform = dict(transform = train_image_transformation, target_transform=train_label_transformation)


KeyError: 'orig_height'

In [4]:
train_batch_size = 1
val_batch_size = 1

In [5]:
def get_data_loaders():
    train_dataset = dataset_cls(DATA_DIR, **train_transform)
    val_dataset = dataset_cls(DATA_DIR, **train_transform)

    train_loader = DataLoader(train_dataset, batch_size=train_batch_size)
    val_loader = DataLoader(val_dataset, batch_size=val_batch_size)

    return (train_loader, val_loader)

In [7]:
train_dataset

Dataset VOCSegmentation
    Number of datapoints: 1464
    Root location: ..\data\PASCALVOC
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(288, 512), interpolation=bilinear, max_size=None, antialias=None)
           )
Target transform: Compose(
                      Resize(size=(288, 512), interpolation=bilinear, max_size=None, antialias=None)
                  )

#### Training step - Basic

In [8]:
import ignite.engine as ign
import ignite.metrics as igm

import segmentation_models_pytorch as smp

from segwork.registry import models_reg

In [9]:
model_name = 'unet'
model_args = dict(
    encoder_name = 'resnet34'
)

device = "cuda" if torch.cuda.is_available() else "cpu"
optimizer_cls = torch.optim.SGD
optimizer_args = dict(
    lr = 0.1,
    momentum = 0.9
)
criterion_args = dict(
    mode = 'multiclass',
    from_logits = False,
    ignore_index = 0                # Background
)
criterion = smp.losses.DiceLoss(**criterion_args)
val_metrics = {"accuracy": igm.Accuracy(), "loss": igm.Loss(criterion)}
log_interval = 10



In [10]:
def run():

    # Get dataloaders
    train_loader, val_loader = get_data_loaders()

    # Get model and place it in device
    model = models_reg.get_instance(model_name, **model_args)
    model.to(device)

    # Optimizer - Can be easily include in a custom registry
    optimizer = optimizer_cls(model.parameters(), **optimizer_args)

    # TRAINER AND EVALUATOR
    trainer = ign.create_supervised_trainer(model, optimizer, criterion, device=device)
    evaluator = ign.create_supervised_evaluator(
    model, metrics=val_metrics, device=device
    )

    # Log results
    _set_events(trainer, evaluator, train_loader, val_loader)
    trainer.run(train_loader, max_epochs=100)

In [11]:
def _set_events(trainer:ign.Engine, evaluator:ign.Engine, train_loader:DataLoader, val_loader:DataLoader):
    @trainer.on(ign.Events.ITERATION_COMPLETED(every=log_interval))
    def log_training_loss(trainer):
        print(f"Epoch[{trainer.state.epoch}] Loss: {trainer.state.output:.2f}")

    @trainer.on(ign.Events.EPOCH_COMPLETED)
    def log_training_results(trainer):
        evaluator.run(train_loader)
        metrics = evaluator.state.metrics
        print(f"Training Results - Epoch: {trainer.state.epoch}  Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")

    @trainer.on(ign.Events.EPOCH_COMPLETED)
    def log_validation_results(trainer):
        evaluator.run(val_loader)
        metrics = evaluator.state.metrics
        print(f"Validation Results - Epoch: {trainer.state.epoch}  Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")

In [12]:
import torch
torch.cuda.empty_cache()

In [13]:
def main():
    run()

main()

Current run is terminating due to exception: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>
Engine run is terminating due to exception: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [44]:
model = models_reg.get_instance(model_name, **model_args)
x = torch.rand(4,3,256, 256)
out = model(x)
out.size()

loss = smp.losses.DiceLoss( mode = 'multiclass')

torch.Size([4, 1, 256, 256])

#### Training step - Custom